In [2]:
%matplotlib inline
import json
import matplotlib.pyplot as plt
import pandas as pd
from pandas.io.json import json_normalize
import time
import numpy as np
import seaborn as sns
import re

In [3]:
path = "executions-bkp-gcloudeu-28-11.jsonl"
raw_data = pd.read_json(path, lines=True)
raw_data.head()

,_id,topic,payload,timestamp
0,{'$oid': '5d274d843299fa6236b8bc28'},data/case/332f02b9-093e-4a2f-8d93-28022c275e8b...,{'responseStamp': 1},2019-07-11 14:53:56.866
1,{'$oid': '5d274d853299fa6236b8bc29'},data/case/332f02b9-093e-4a2f-8d93-28022c275e8b...,"{'name': 'Zombie Venom 09', 'source': '# 1 (ca...",2019-07-11 14:53:57.587
2,{'$oid': '5d274e633299fa6236b8bc2a'},data/case/00f14675-854c-4c58-95eb-8dd414f525d3...,{'responseStamp': 1},2019-07-11 14:57:39.979
3,{'$oid': '5d274e643299fa6236b8bc2b'},data/case/00f14675-854c-4c58-95eb-8dd414f525d3...,"{'name': 'Zombie Venom 08', 'source': '# 1 (ca...",2019-07-11 14:57:40.069
4,{'$oid': '5d274e643299fa6236b8bc2c'},data/theme/zombie.note/get,{'responseStamp': 2},2019-07-11 14:57:40.085


In [4]:
nl_experiment = raw_data.loc[lambda df: df.timestamp > '2019-11-28 7:30:00']
nl_experiment.to_json('nl_experiment1.json', orient='records')

In [5]:
def reconstruct_users_track(log):
    users_tracks = dict()
    for index, row in log.iterrows():
        payload = row.payload
        if "track" in payload and "userid" in payload['track']:
            user = payload['track']['userid'] 
            if user in users_tracks:
                users_tracks[user].append(row)
            else:
                users_tracks.update({user: [row]})
    return users_tracks

In [7]:
nl_experiment.head(10)

,_id,topic,payload,timestamp
22113,{'$oid': '5ddf8655cd683579f92534ee'},data/user/login,"{'email': 'A33@ds4h.org', 'password': 'a33', '...",2019-11-28 08:33:25.387
22114,{'$oid': '5ddf8655cd683579f92534ef'},data/user/login/response/2,"{'userid': 93, 'token': 'eyJhbGciOiJIUzI1NiIsI...",2019-11-28 08:33:25.551
22115,{'$oid': '5ddf8655cd683579f92534f0'},data/case/*/list,"{'filterBy': 'user', 'filter': 93, 'responseSt...",2019-11-28 08:33:25.614
22116,{'$oid': '5ddf8655cd683579f92534f1'},data/case/*/list/response/3,[{'id': 'a46c78ec-9973-4792-a4ec-d9a9ef0e4978'...,2019-11-28 08:33:25.729
22117,{'$oid': '5ddf8655cd683579f92534f2'},data/case/a46c78ec-9973-4792-a4ec-d9a9ef0e4978...,{'responseStamp': 4},2019-11-28 08:33:25.754
22118,{'$oid': '5ddf8655cd683579f92534f3'},data/case/a46c78ec-9973-4792-a4ec-d9a9ef0e4978...,"{'name': 'EKG-First', 'source': 'EKG-Descripti...",2019-11-28 08:33:25.883
22119,{'$oid': '5ddf8656cd683579f92534f4'},knot/<</navigate,{},2019-11-28 08:33:26.004
22120,{'$oid': '5ddf8656cd683579f92534f5'},2019-11-28T08:33:25.342Z-980cae4f-15d1-9d08-11...,"{'parameter': 'EKG first', 'track': {'userid':...",2019-11-28 08:33:26.033
22121,{'$oid': '5ddf8656cd683579f92534f6'},2019-11-28T08:33:25.342Z-980cae4f-15d1-9d08-11...,"{'track': {'userid': 93, 'caseid': 'a46c78ec-9...",2019-11-28 08:33:26.036
22122,{'$oid': '5ddf8656cd683579f92534f7'},2019-11-28T08:33:25.342Z-980cae4f-15d1-9d08-11...,"{'track': {'userid': 93, 'caseid': 'a46c78ec-9...",2019-11-28 08:33:26.082


In [8]:
nl_experiment

,_id,topic,payload,timestamp
22113,{'$oid': '5ddf8655cd683579f92534ee'},data/user/login,"{'email': 'A33@ds4h.org', 'password': 'a33', '...",2019-11-28 08:33:25.387
22114,{'$oid': '5ddf8655cd683579f92534ef'},data/user/login/response/2,"{'userid': 93, 'token': 'eyJhbGciOiJIUzI1NiIsI...",2019-11-28 08:33:25.551
22115,{'$oid': '5ddf8655cd683579f92534f0'},data/case/*/list,"{'filterBy': 'user', 'filter': 93, 'responseSt...",2019-11-28 08:33:25.614
22116,{'$oid': '5ddf8655cd683579f92534f1'},data/case/*/list/response/3,[{'id': 'a46c78ec-9973-4792-a4ec-d9a9ef0e4978'...,2019-11-28 08:33:25.729
22117,{'$oid': '5ddf8655cd683579f92534f2'},data/case/a46c78ec-9973-4792-a4ec-d9a9ef0e4978...,{'responseStamp': 4},2019-11-28 08:33:25.754
...,...,...,...,...
104931,{'$oid': '5ddf9024cd683579f9267870'},2019-11-28T09:15:15.570Z-c73e9cbd-85e8-e63d-93...,"{'responseStamp': 8, 'track': {'userid': 116, ...",2019-11-28 09:15:16.547
104932,{'$oid': '5ddf9024cd683579f9267871'},2019-11-28T09:15:15.570Z-c73e9cbd-85e8-e63d-93...,"{'content': '<div class=""styt-main-frame""> ...",2019-11-28 09:15:16.584
104933,{'$oid': '5ddf9024cd683579f9267872'},2019-11-28T09:15:15.570Z-c73e9cbd-85e8-e63d-93...,"{'track': {'userid': 116, 'caseid': '9b1651f3-...",2019-11-28 09:15:16.624
104934,{'$oid': '5ddf9025cd683579f9267873'},data/case/*/list,{'responseStamp': 2},2019-11-28 09:15:17.504


In [9]:
users_track = reconstruct_users_track(nl_experiment)

In [10]:
users_track[93][0]['payload']['parameter']

'EKG first'

In [11]:
def split_groups(users_track):
    group_A = dict()
    group_B = dict()
    
    for user in users_track.keys():
        for log in users_track[user]:
            try:
                if log['payload']['parameter'] == 'Description first':
                    group_A[user] = users_track[user]
                if log['payload']['parameter'] == 'EKG first':
                    group_B[user] = users_track[user]
            except:
                pass
    return group_A, group_B

In [12]:
group_A, group_B = split_groups(users_track)
print("Users Group A - Description First")
print(sorted(list(group_A.keys())))
print("Users Group B - EKG First")
print(sorted(list(group_B.keys())))

Users Group A - Description First
[1, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 87]
Users Group B - EKG First
[86, 87, 88, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 115, 116]


In [13]:
print(len(list(group_A.keys())))
print(len(list(group_B.keys())))

26
29


In [58]:
def users_answers(users_track):
    
    users_answers = dict()
    
    for user in users_track.keys():
        users_answers[user] = pd.DataFrame(columns=["Variable", "Answer", "CaseId"])
        
        for log in users_track[user]:
            if re.search('/var/[\w+.]+/set', log['topic']):
                
                var_topic = re.search('/var/[\w+.]+/set', log['topic']).group()
                answer = log['payload']['content']
                case_id = log['payload']['track']['caseid']
                row = {"Variable": var_topic, "Answer": answer, "CaseId": case_id}
                users_answers[user] = users_answers[user].append(row, ignore_index=True)
                
    return users_answers

In [61]:
answers_groupA = users_answers(group_A)
answers_groupB = users_answers(group_B)

In [113]:
def clean_group(df_group):
    for key in df_group:
        print(type(df_group[key]))

In [165]:
def clean_variables_track(answers_track, columns):
    
    users_answers_clean = dict()
    
    for user in answers_track.keys():
        cleaned_track = pd.DataFrame(columns=["Variable", "Answer", "CaseId"])
        group_track = answers_track[user].groupby(columns)
        
        for name, group in group_track:
            cleaned_track = cleaned_track.append(group_track.get_group(name).iloc[-1])
        
        users_answers_clean[user] = cleaned_track.sort_values(by=["CaseId","Variable"], ascending=False)
    
    return users_answers_clean

In [168]:
groupA_clean = clean_variables_track(answers_groupA, columns=["Variable", "CaseId"])
groupB_clean = clean_variables_track(answers_groupB, columns=["Variable", "CaseId"])

In [187]:
groupA_clean[61]

,Variable,Answer,CaseId
70,/var/EKG_Cycle.Generate_hypothesis.hypothesis/set,Acs,ec4be4fd-4e48-4e84-ba8e-36c53b415698
71,/var/EKG_Cycle.Generate_hypothesis.confidence/set,84,ec4be4fd-4e48-4e84-ba8e-36c53b415698
68,/var/Description_Cycle.Generate_hypothesis.hyp...,Acs,ec4be4fd-4e48-4e84-ba8e-36c53b415698
69,/var/Description_Cycle.Generate_hypothesis.con...,75,ec4be4fd-4e48-4e84-ba8e-36c53b415698
118,/var/EKG_Cycle.Generate_hypothesis.hypothesis/set,Acs 3vd,b90056f9-e5e2-4281-bcfe-75ab62033b2b
119,/var/EKG_Cycle.Generate_hypothesis.confidence/set,89,b90056f9-e5e2-4281-bcfe-75ab62033b2b
116,/var/Description_Cycle.Generate_hypothesis.hyp...,Acs,b90056f9-e5e2-4281-bcfe-75ab62033b2b
117,/var/Description_Cycle.Generate_hypothesis.con...,80,b90056f9-e5e2-4281-bcfe-75ab62033b2b
106,/var/EKG_Cycle.Generate_hypothesis.hypothesis/set,Acs,a775bf21-b15c-44d1-a120-61611bc7960c
105,/var/EKG_Cycle.Generate_hypothesis.confidence/set,90,a775bf21-b15c-44d1-a120-61611bc7960c


In [188]:
groupB_clean[86]

,Variable,Answer,CaseId
49,/var/EKG_Cycle.Generate_hypothesis.hypothesis/set,Doorgemaakt infarct.,ec4be4fd-4e48-4e84-ba8e-36c53b415698
50,/var/EKG_Cycle.Generate_hypothesis.confidence/set,19,ec4be4fd-4e48-4e84-ba8e-36c53b415698
48,/var/Description_Cycle.Generate_hypothesis.hyp...,Doorgemaakt infarct,ec4be4fd-4e48-4e84-ba8e-36c53b415698
47,/var/Description_Cycle.Generate_hypothesis.con...,90,ec4be4fd-4e48-4e84-ba8e-36c53b415698
93,/var/EKG_Cycle.Generate_hypothesis.hypothesis/set,"Sinus tachycardia, nonspecific ST changes",b90056f9-e5e2-4281-bcfe-75ab62033b2b
94,/var/EKG_Cycle.Generate_hypothesis.confidence/set,70,b90056f9-e5e2-4281-bcfe-75ab62033b2b
92,/var/Description_Cycle.Generate_hypothesis.hyp...,IAP,b90056f9-e5e2-4281-bcfe-75ab62033b2b
88,/var/Description_Cycle.Generate_hypothesis.con...,86,b90056f9-e5e2-4281-bcfe-75ab62033b2b
83,/var/EKG_Cycle.Generate_hypothesis.hypothesis/set,Normal,a775bf21-b15c-44d1-a120-61611bc7960c
82,/var/EKG_Cycle.Generate_hypothesis.confidence/set,100,a775bf21-b15c-44d1-a120-61611bc7960c


In [183]:
def save_as_xlsx(users_dict, path, filename_p=""):
    
    filename = path + filename_p
    
    for user in users_dict.keys():
        
        filename += str(user) + ".xlsx"
        users_dict[user].to_excel(filename, index=False)
        filename = path + filename_p

In [184]:
path = "/home/marcosf/git/harena-analytics/resources/nl-experiment1/Tables/GroupA/"
save_as_xlsx(groupA_clean, path)

In [185]:
path_B = "/home/marcosf/git/harena-analytics/resources/nl-experiment1/Tables/GroupB/"
save_as_xlsx(groupB_clean, path_B)